In [ ]:
# wine_descriptive_stats.py
# Thống kê mô tả bộ dữ liệu Wine Quality (UCI)

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# =========================
# Cấu hình
# =========================
# Đổi tên file tại đây: "winequality-red.csv" hoặc "winequality-white.csv"
CSV_PATH = "/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv"
OUTPUT_DIR = "figs"

NUM_COLS = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]
TARGET_COL = "quality"

# =========================
# Helper
# =========================
def nan_mode(series: pd.Series):
    """Trả mode an toàn với NaN (lấy mode đầu tiên nếu có)."""
    vals = series.dropna().mode()
    return vals.iat[0] if len(vals) else np.nan

def iqr_bounds(series: pd.Series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return q1, q3, iqr, lower, upper

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

# =========================
# 1) Nạp dữ liệu & ép kiểu
# =========================
df = pd.read_csv(CSV_PATH)

# Đảm bảo các cột đúng tên kỳ vọng; nếu UCI có dấu ";" thì pandas đã tách đúng.
# Ép kiểu numeric cho toàn bộ cột số (an toàn)
for c in NUM_COLS + [TARGET_COL]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# =========================
# 2) Kiểm tra thiếu dữ liệu & xử lý
# =========================
print("== Missing values per column ==")
print(df[NUM_COLS + [TARGET_COL]].isna().sum(), "\n")

# Điền thiếu bằng median cho cột số (giữ phân bố hợp lý)
for c in NUM_COLS:
    med = df[c].median(skipna=True)
    df[c] = df[c].fillna(med)

# Nếu quality có NaN (hiếm), có thể loại bỏ
df = df.dropna(subset=[TARGET_COL]).copy()

# =========================
# 3) Thống kê mô tả toàn cục
# =========================
desc = df[NUM_COLS].describe(percentiles=[0.25, 0.5, 0.75]).T
desc["range"] = df[NUM_COLS].max() - df[NUM_COLS].min()
desc["IQR"] = desc["75%"] - desc["25%"]
# skew/kurtosis
desc["skew"] = df[NUM_COLS].skew()
desc["kurtosis"] = df[NUM_COLS].kurt()

# mode từng cột
modes = {c: nan_mode(df[c]) for c in NUM_COLS}
desc["mode"] = pd.Series(modes)

print("== Descriptive statistics (toàn bộ dữ liệu) ==")
print(desc.round(3), "\n")

# Percentiles tùy biến (5, 25, 50, 75, 95)
percentiles = [5, 25, 50, 75, 95]
perc_table = pd.DataFrame(
    {c: np.percentile(df[c], percentiles, method="linear") for c in NUM_COLS},
    index=[f"P{p}" for p in percentiles],
)
print("== Selected percentiles ==")
print(perc_table.round(3).T, "\n")

# =========================
# 4) Phát hiện ngoại lai theo IQR
# =========================
outlier_report = []
for c in NUM_COLS:
    q1, q3, iqr, lo, hi = iqr_bounds(df[c])
    mask = (df[c] < lo) | (df[c] > hi)
    outlier_report.append((c, mask.sum(), float(lo), float(hi)))
outlier_df = pd.DataFrame(outlier_report, columns=["feature", "n_outliers", "lower", "upper"])
print("== Outliers theo IQR (count / bounds) ==")
print(outlier_df.sort_values("n_outliers", ascending=False), "\n")

# =========================
# 5) Thống kê theo từng mức quality
# =========================
# =========================
# 5) Thống kê theo từng mức quality (FIX)
# =========================

def q1(s): 
    return s.quantile(0.25)
def q3(s): 
    return s.quantile(0.75)

# Đặt tên hàm để khi flatten cột sẽ đẹp
q1.__name__ = "q1"
q3.__name__ = "q3"

# Cách tương thích rộng: truyền list các hàm vào .agg()
grouped = (
    df.groupby(TARGET_COL)[NUM_COLS]
      .agg(['mean', 'median', 'std', q1, q3])
)

# (Tuỳ chọn) Làm phẳng MultiIndex columns: "feature_stat"
grouped.columns = [f"{col}_{stat if isinstance(stat, str) else stat.__name__}" 
                   for col, stat in grouped.columns]

grouped = grouped.round(3)
print("== Descriptive by quality (mean/median/std/Q1/Q3) ==")
print(grouped, "\n")

ensure_dir(OUTPUT_DIR)

# 6.0 Phân bố nhãn quality
plt.figure()
df[TARGET_COL].value_counts().sort_index().plot(kind="bar")
plt.title("Quality distribution")
plt.xlabel("quality"); plt.ylabel("count")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "quality_distribution.png"), dpi=130)
plt.close()

# 6.1 Histogram cho từng biến
for c in NUM_COLS:
    plt.figure()
    df[c].plot(kind="hist", bins=40, edgecolor="black")
    plt.title(f"{c} — Histogram")
    plt.xlabel(c); plt.ylabel("count")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f"hist_{c.replace(' ', '_')}.png"), dpi=130)
    plt.close()

# 6.2 Boxplot cho tất cả biến số
plt.figure(figsize=(10, 5))
df[NUM_COLS].boxplot(rot=45)
plt.title("Boxplot — All numeric features")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "boxplot_all.png"), dpi=130)
plt.close()

# 6.3 Heatmap tương quan Pearson
corr = df[NUM_COLS + [TARGET_COL]].corr(method="pearson")
plt.figure(figsize=(7, 6))
im = plt.imshow(corr.values, interpolation="nearest")
plt.title("Correlation heatmap (Pearson)")
plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha="right")
plt.yticks(range(len(corr.index)), corr.index)
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "corr_heatmap.png"), dpi=130)
plt.close()

# 6.4 Scatter: alcohol vs. quality (trend thô)
plt.figure()
plt.scatter(df["alcohol"], df["quality"], alpha=0.4)
plt.title("Alcohol vs Quality")
plt.xlabel("alcohol"); plt.ylabel("quality")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "scatter_alcohol_quality.png"), dpi=130)
plt.close()

# 6.5 (Tùy chọn) Biểu đồ theo nhóm quality cho vài biến quan trọng (cần seaborn)
try:
    import seaborn as sns
    sns.set_theme(style="whitegrid")
    # alcohol by quality
    plt.figure(figsize=(8,5))
    sns.boxplot(data=df, x="quality", y="alcohol")
    plt.title("Alcohol by quality")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, "box_alcohol_by_quality.png"), dpi=130)
    plt.close()

    # volatile acidity by quality
    plt.figure(figsize=(8,5))
    sns.violinplot(data=df, x="quality", y="volatile acidity", inner="quartile", cut=0)
    plt.title("Volatile acidity by quality")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, "violin_va_by_quality.png"), dpi=130)
    plt.close()
except Exception as e:
    print("[INFO] seaborn not available or failed to plot:", e)

print(f"Hình đã lưu tại ./{OUTPUT_DIR}/")

== Missing values per column ==
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64 

== Descriptive statistics (toàn bộ dữ liệu) ==
                       count    mean     std    min     25%     50%     75%  \
fixed acidity         1599.0   8.320   1.741  4.600   7.100   7.900   9.200   
volatile acidity      1599.0   0.528   0.179  0.120   0.390   0.520   0.640   
citric acid           1599.0   0.271   0.195  0.000   0.090   0.260   0.420   
residual sugar        1599.0   2.539   1.410  0.900   1.900   2.200   2.600   
chlorides             1599.0   0.087   0.047  0.012   0.070   0.079   0.090   
free sulfur dioxide   1599.0  15.875  10.460  1.000   7.000  14.000  21.000   
total sulfur dioxide  1599.0  46.468  32.